In [ ]:
# !pip install geemap

In [1]:
import os
import ee
import geemap

In [3]:
!python --version

Python 3.10.0


Upgrade geemap to the latest version

In [2]:
# geemap.update_package()

In [2]:
Map = geemap.Map()
Map

Enter verification code: 4/1AX4XfWgitQVAMEZZulrKfU66KN_rLVnxXcojm7Lsd6nn2J6WRVadqTMgD4c

Successfully saved authorization token.


AttributeError: module 'collections' has no attribute 'Callable'

In [ ]:
dataset_izh = ee.ImageCollection('COPERNICUS/S2_SR')
Map.addLayer(dataset_izh, {}, "COPERNICUS/S2_SR")

In [ ]:
SentinelA2 = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2019-01-01', '2019-01-05')
image = SentinelA2.toBands()

In [ ]:
Map.addLayer(image,{'bands':['B1', 'B2', 'B3']})

In [ ]:
in_shp = '2021 T3/Training_data_generation/Python/Bill/AllFarmDams.shp'

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'FarmDams')

In [ ]:
work_dir = os.path.expanduser('~/Downloads/')
in_shp = os.path.join(work_dir, 'AllFarmDams.shp')
if not os.path.exists(in_shp):
    data_url = 'https://github.com/BillHuynh/DataScienceProjects/raw/e9526a145633be5ac3535db638c328f9bd191fc1/AllFarmDams.zip'
    geemap.download_from_url(data_url, out_dir=work_dir )

In [ ]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
Map.addLayer(landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7')
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 1)

Download sample data

In [ ]:
work_dir = os.path.expanduser('~/Downloads')
in_shp = os.path.join(work_dir, 'us_cities.shp')
if not os.path.exists(in_shp):
    data_url = 'https://github.com/giswqs/data/raw/main/us/us_cities.zip'
    geemap.download_from_url(data_url, out_dir=work_dir )

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'Cities')

Export pixel values as a shapefile

In [ ]:
out_shp = os.path.join(work_dir, 'dem.shp')
geemap.extract_values_to_points(in_fc, dem, out_shp)

Export pixel values as a csv

In [ ]:
out_csv = os.path.join(work_dir, 'landsat.csv')
geemap.extract_values_to_points(in_fc, landsat7, out_csv)

In [ ]:
js_snippet = '''
var FarmDams = ee.FeatureCollection("users/rishabhjindal45/AllFarmDams"),
    Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR"),
    Region = 
    /* color: #d63000 */
    /* shown: false */
    ee.Geometry.Polygon(
        [[[148.9405054323802, -32.697231431596926],
          [145.6006616823802, -33.36040933369022],
          [143.9307398073802, -34.671670280678136],
          [143.1397241823802, -39.23443034959701],
          [144.8975366823802, -39.506209455992895],
          [147.9737085573802, -39.506209455992895],
          [149.5557398073802, -38.96159444142003],
          [152.1924585573802, -35.891327255966466],
          [152.8076929323802, -34.382032369403966]]]);

// Choose here the polygons to analyse
var myPoly = FarmDams;

// Choose here the satellite data
var mySat = Sentinel2A;

 // Choose dates
var StartDate = '2019-01-01';
var EndDate = '2020-01-01';

// Output name (Water)
var SaveWater = 'FarmDams_water_AllBandsAllDates_level2A';

// Output name (Veg)
var SaveVeg = 'FarmDams_vegetation_AllBandsAllDates_level2A';

////////////////////////////////////////


function zonalStats(ic, fc, params) {
  // Initialize internal params dictionary.
  var _params = {
    reducer: ee.Reducer.mean(),
    scale: null,
    crs: null,
    bands: null,
    bandsRename: null,
    imgProps: null,
    imgPropsRename: null,
    datetimeName: 'datetime',
    datetimeFormat: 'YYYY-MM-dd HH:MM:ss'
  };

  // Replace initialized params with provided params.
  if (params) {
    for (var param in params) {
      _params[param] = params[param] || _params[param];
    }
  }

  // Set default parameters based on an image representative.
  var imgRep = ic.first();
  var nonSystemImgProps = ee.Feature(null)
    .copyProperties(imgRep).propertyNames();
  if (!_params.bands) _params.bands = imgRep.bandNames();
  if (!_params.bandsRename) _params.bandsRename = _params.bands;
  if (!_params.imgProps) _params.imgProps = nonSystemImgProps;
  if (!_params.imgPropsRename) _params.imgPropsRename = _params.imgProps;

  // Map the reduceRegions function over the image collection.
  var results = ic.map(function(img) {
    // Select bands (optionally rename), set a datetime & timestamp property.
    img = ee.Image(img.select(_params.bands, _params.bandsRename))
      .set(_params.datetimeName, img.date().format(_params.datetimeFormat))
      .set('timestamp', img.get('system:time_start'));

    // Define final image property dictionary to set in output features.
    var propsFrom = ee.List(_params.imgProps)
      .cat(ee.List([_params.datetimeName, 'timestamp']));
    var propsTo = ee.List(_params.imgPropsRename)
      .cat(ee.List([_params.datetimeName, 'timestamp']));
    var imgProps = img.toDictionary(propsFrom);

    // Subset points that intersect the given image.
    var fcSub = fc.filterBounds(img.geometry());

    // Reduce the image by regions.
    return img.reduceRegions({
      collection: fcSub,
      reducer: _params.reducer,
      scale: _params.scale,
      crs: _params.crs
    })
    // Add metadata to each feature.
    .map(function(f) {
      return f.set(imgProps);
    })
    .map(function(feature) {
    var ndvi = ee.List([feature.get('quality_check'), -9999])
      .reduce(ee.Reducer.firstNonNull())
    return feature.set({'quality_check': ndvi, 'imageID': img.id()})
    })
    
  }).flatten().filter(ee.Filter.notNull(_params.bandsRename));

  return results;
  return fcSub;

}

var modisCol = ee.ImageCollection(mySat
    .filterBounds(Region)
    .filterDate(StartDate, EndDate)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
    .select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B8A', 'B11', 'B12')
  );

//print(modisCol)

// Define parameters for the zonalStats function.
var params = {
  scale: 0,
  bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B8A', 'B11', 'B12'],
  bandsRename: ['Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12'],
  datetimeName: 'date',
  datetimeFormat: 'YYYY-MM-dd'
};




// Load the function to add a buffer to each point
// This is not needed when dealing with POLYGONS instead of POINTS
function bufferPoints(radius, bounds) {
  return function(pt) {
    pt = ee.Feature(pt);
    return bounds ? pt.buffer(radius).bounds() : pt.buffer(radius);
  };
}




///////////////////////////
// ANALYSE THE FARM DAMS //
///////////////////////////

// Remove 5 meters of buffer at each polygon to make sure we get the water
//var PolyWater = ANUFarms.map(bufferPoints(-5, false));
var PolyWater = myPoly.map(bufferPoints(-5, false));
print(PolyWater);


// Extract zonal statistics per point per image and calculating the area of polygon in m2.
var PolyWater_Stats = zonalStats(modisCol,PolyWater, params).map(function(f) {
          var stateArea = f.geometry().area()
          return f.set('Area_in_m2', ee.Number(stateArea));
    });
//print(PolyWater_Stats);


// Save water stats
Export.table.toDrive({
  collection: PolyWater_Stats,
  folder: 'GEE',
  description: SaveWater,
  fileFormat: 'CSV',
  selectors:['Ref','Sites','date','Band_1', 'Band_2', 'Band_3', 'Band_4','Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12','Area_in_m2']
});



/////////////////////////////////
// ANALYSE THE SURROUNDING VEG //
/////////////////////////////////


// Add 30 meters of buffer at each polygon for the vegetation
//var PolyLarge = ANUFarms.map(bufferPoints(30, false));

// Remove the water section
// See https://gis.stackexchange.com/questions/300725/difference-between-two-polygons-in-earth-engine
// See https://gis.stackexchange.com/questions/246161/compute-centroids-and-buffers-for-multiple-polygons-in-google-earth-engine
// This option doesn't work because you are dealing with a collection of polygons
// var PolyVeg = PolyLarge.difference(ANUFarms);
// You need to MAP over multiple polygons

var PolyDiff = function(Poly) {
  var PolyLarge = Poly.buffer(50); // Create the large polygon
  return PolyLarge.difference(Poly);
};

var PolyVeg = myPoly.map(PolyDiff);

//print(PolyVeg);


// Extract zonal statistics per point per image and calculating the area of polygon in m2.
var PolyVeg_Stats = zonalStats(modisCol,PolyVeg, params).map(function(f) {
          var stateArea = f.geometry().area()
          return f.set('Area_in_m2', ee.Number(stateArea));
    });
    
print(PolyVeg_Stats);


// Save vegetation stats
Export.table.toDrive({
  collection: PolyVeg_Stats,
  folder: 'GEE',
  description: SaveVeg,
  fileFormat: 'CSV',
  selectors:['Ref','Sites','date','Band_1', 'Band_2', 'Band_3', 'Band_4','Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12','Area_in_m2']
});



//////////
// PLOT //
//////////

Map.setCenter(143.96306, -37.74696, 12);
//Map.addLayer(modisCol, {min: 3000, max: 3500});
Map.addLayer(PolyWater,{} , 'Water');
Map.addLayer(PolyVeg, {} , 'Vegetation');

'''

In [ ]:
geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

In [ ]:
import ee
import geemap

Map = geemap.Map()

FarmDams = ee.FeatureCollection("users/rishabhjindal45/AllFarmDams"),
    Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR"),
    Region =

    # shown: False #
    ee.Geometry.Polygon(
        [[[148.9405054323802, -32.697231431596926],
          [145.6006616823802, -33.36040933369022],
          [143.9307398073802, -34.671670280678136],
          [143.1397241823802, -39.23443034959701],
          [144.8975366823802, -39.506209455992895],
          [147.9737085573802, -39.506209455992895],
          [149.5557398073802, -38.96159444142003],
          [152.1924585573802, -35.891327255966466],
          [152.8076929323802, -34.382032369403966]]])

# Choose here the polygons to analyse
myPoly = FarmDams

# Choose here the satellite data
mySat = Sentinel2A

 # Choose dates
StartDate = '2019-01-01'
EndDate = '2020-01-01'

# Output name (Water)
SaveWater = 'FarmDams_water_AllBandsAllDates_level2A'

# Output name (Veg)
SaveVeg = 'FarmDams_vegetation_AllBandsAllDates_level2A'

####################

def zonalStats(ic, fc, params):
  # Initialize internal params dictionary.
  _params = {
    'reducer': ee.Reducer.mean(),
    'scale': {},
    'crs': {},
    'bands': {},
    'bandsRename': {},
    'imgProps': {},
    'imgPropsRename': {},
    'datetimeName': 'datetime',
    'datetimeFormat': 'YYYY-MM-dd HH:'MM':ss'
  }

  # Replace initialized params with provided params.
  if (params) {
    for param in params:
      _params[param] = params[param] || _params[param]

  }

  # Set default parameters based on an image representative.
  imgRep = ic.first()
  nonSystemImgProps = ee.Feature({}) \
    .copyProperties(imgRep).propertyNames()
  if (!_params.bands) _params.bands = imgRep.bandNames()
  if (!_params.bandsRename) _params.bandsRename = _params.bands
  if (!_params.imgProps) _params.imgProps = nonSystemImgProps
  if (!_params.imgPropsRename) _params.imgPropsRename = _params.imgProps

  # Map the reduceRegions function over the image collection.

def func_dyw(img):
    # Select bands (optionally rename), set a datetime & timestamp property.
    img = ee.Image(img.select(_params.bands, _params.bandsRename)) \
      .set(_params.datetimeName, img.date().format(_params.datetimeFormat)) \
      .set('timestamp', img.get('system:time_start'))

    # Define final image property dictionary to set in output features.
    propsFrom = ee.List(_params.imgProps) \
      .cat(ee.List([_params.datetimeName, 'timestamp']))
    propsTo = ee.List(_params.imgPropsRename) \
      .cat(ee.List([_params.datetimeName, 'timestamp']))
    imgProps = img.toDictionary(propsFrom)

    # Subset points that intersect the given image.
    fcSub = fc.filterBounds(img.geometry())

    # Reduce the image by regions.
    return img.reduceRegions({
      'collection': fcSub,
      'reducer': _params.reducer,
      'scale': _params.scale,
      'crs': _params.crs
    }) \
    .map(function(f) {
      return f.set(imgProps)
    }) \
    .map(function(feature) {
    ndvi = ee.List([feature.get('quality_check'), -9999]) \
      .reduce(ee.Reducer.firstNonNull())
    return feature.set({'quality_check': ndvi, 'imageID': img.id()})
    })

  results = ic.map(func_dyw
).flatten().filter(ee.Filter.NotNull(_params.bandsRename))

).flatten().filter(ee.Filter.NotNull(_params.bandsRename))

  return results
  return fcSub

modisCol = ee.ImageCollection(mySat \
    .filterBounds(Region) \
    .filterDate(StartDate, EndDate) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
    .select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B8A', 'B11', 'B12')
  )

#print(modisCol)

# Define parameters for the zonalStats function.
params = {
  'scale': 0,
  'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B8A', 'B11', 'B12'],
  'bandsRename': ['Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12'],
  'datetimeName': 'date',
  'datetimeFormat': 'YYYY-MM-dd'
}

# Load the function to add a buffer to each point
# This is not needed when dealing with POLYGONS instead of POINTS
def bufferPoints(radius, bounds):
  return function(pt) {
    pt = ee.Feature(pt)
    'return bounds ? pt.buffer(radius).bounds()' : pt.buffer(radius)
  }

#############/
# ANALYSE THE FARM DAMS #
#############/

# Remove 5 meters of buffer at each polygon to make sure we get the water
#PolyWater = ANUFarms.map(bufferPoints(-5, False))
PolyWater = myPoly.map(bufferPoints(-5, False))
print(PolyWater)

# Extract zonal statistics per point per image and calculating the area of polygon in m2.

def func_seq(f):
          stateArea = f.geometry().area()
          return f.set('Area_in_m2', ee.Number(stateArea))

PolyWater_Stats = zonalStats(modisCol,PolyWater, params).map(func_seq)

#print(PolyWater_Stats)

# Save water stats
Export.table.toDrive({
  'collection': PolyWater_Stats,
  'folder': 'GEE',
  'description': SaveWater,
  'fileFormat': 'CSV',
  'selectors':['Ref','Sites','date','Band_1', 'Band_2', 'Band_3', 'Band_4','Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12','Area_in_m2']
})

################/
# ANALYSE THE SURROUNDING VEG #
################/

# Add 30 meters of buffer at each polygon for the vegetation
#PolyLarge = ANUFarms.map(bufferPoints(30, False))

# Remove the water section
# See https:#gis.stackexchange.com/questions/300725/difference-between-two-polygons-in-earth-engine
# See https:#gis.stackexchange.com/questions/246161/compute-centroids-and-buffers-for-multiple-polygons-in-google-earth-engine
# This option doesn't work because you are dealing with a collection of polygons
# PolyVeg = PolyLarge.difference(ANUFarms)
# You need to MAP over multiple polygons

def PolyDiff(Poly):
  PolyLarge = Poly.buffer(50); # Create the large polygon
  return PolyLarge.difference(Poly)

PolyVeg = myPoly.map(PolyDiff)

#print(PolyVeg)

# Extract zonal statistics per point per image and calculating the area of polygon in m2.

def func_jlx(f):
          stateArea = f.geometry().area()
          return f.set('Area_in_m2', ee.Number(stateArea))

PolyVeg_Stats = zonalStats(modisCol,PolyVeg, params).map(func_jlx)

print(PolyVeg_Stats)

# Save vegetation stats
Export.table.toDrive({
  'collection': PolyVeg_Stats,
  'folder': 'GEE',
  'description': SaveVeg,
  'fileFormat': 'CSV',
  'selectors':['Ref','Sites','date','Band_1', 'Band_2', 'Band_3', 'Band_4','Band_5', 'Band_6', 'Band_7','Band_8','Band_9','Band_B8A','Band_11','Band_12','Area_in_m2']
})

#####
# PLOT #
#####

Map.setCenter(143.96306, -37.74696, 12)
#Map.addLayer(modisCol, {min: 3000, max: 3500})
Map.addLayer(PolyWater,{} , 'Water')
Map.addLayer(PolyVeg, {} , 'Vegetation')
Map
